In [1]:
%pip install transformers 
%pip install sklearn
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=f606a9286ae9499271051b4aeae39202c9e2b74a85766cbf757faead7a7a469c
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 115 kB 85.2 MB/s 
     |████████████████████████████████| 212 kB 81.1 MB/s 
     |████████████████████████████████| 127 kB 87.2 MB/s 
  Attempting

In [2]:
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset, load_metric
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding,get_scheduler
from torch.optim import AdamW

from spacy import tokenizer
from spacy.lang.en import English
import torch
from torch.utils.data import DataLoader
from utils import RoBERT_Model,ToBERT_Model
import json
import matplotlib.pyplot as plt

In [13]:
from utils import AssessData

In [3]:
accuracy = load_metric('accuracy'); f1_score = load_metric('f1'); perplexity = load_metric('perplexity')

In [4]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/600000 [00:00<?, ? examples/s]

Dataset hyperpartisan_news_detection downloaded and prepared to /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

dict_keys(['train', 'validation'])

In [42]:
#change to bypublisher because byarticle has no test set.
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = 32)
hyperpartisan_test = hyperpartisan_valid.shuffle(seed = 32)

With huggingface

DOCUMENTS LESS THAN 300

In [17]:
#Should work once then modularize

max_input_length = 300 #Vary after obtaining end to end result
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, max_length = max_input_length)
overlap = {"side":"both", "number": 50}; f1_score = load_metric("f1"); precision = load_metric("precision"); recall = load_metric("recall")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #to pad sequences


def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=max_input_length)

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {"f1" :f1_score.compute(predictions = predictions, references = label, average = 'weighted'),
            "precision" : precision.compute(predictions = predictions, references = label, average = 'weighted'),
            "recall": recall.compute(predictions = predictions, references = label, average = 'weighted')}

def hp_space(trial):

  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.0001,0.0005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)
  }

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [43]:
hyperpartisan_train = hyperpartisan_train.select(range(2000)) #inconclusive because of nature of dataset
hyperpartisan_test = hyperpartisan_test.select(range(2000))

In [44]:
int_labels_train = {i: 1 if x == True else 0 for i,x in enumerate(hyperpartisan_train['hyperpartisan'])}
int_labels_test = {i: 1 if x == True else 0 for i,x in enumerate(hyperpartisan_test['hyperpartisan'])}

In [ ]:
hyperpartisan_train = hyperpartisan_train.map(tokenize)
hyperpartisan_test = hyperpartisan_test.map(tokenize)

EFFECT OF FINETUNING (with varying number of training files)

In [45]:
hyp_train = {i:x for i,x in enumerate(hyperpartisan_train['text'])}
hyp_test = {i:x for i,x in enumerate(hyperpartisan_test['text'])}

In [46]:
hyperpartisan_train = hyperpartisan_train.add_column("labels", int_labels_train.values())
hyperpartisan_test = hyperpartisan_test.add_column("labels", int_labels_test.values())

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
#Problems with finetuning with limited number of training files, is training set is unbalanced
#Less knowledge for neural network to learn from 
#Why not investigate the definition of what a chunk is, and use different pretraining objective e.g BERT,Roberta
#Problem with finetuning with long document is document input are limited in length, therefore learning across text documents is unbalanced

#Only way around finetuning on text document is to carefully analyse the training set and long documents.

In [ ]:
#Insight into the distribution of the dataset
plt.hist(hyperpartisan_train['labels'])

In [ ]:
from sklearn.model_selection import KFold

In [47]:
hyp_train = AssessData(hyp_train, dict())
hyp_test = AssessData(hyp_test, dict())

In [48]:
number = range(50,150,50); _per_segment = range(200,400,100); max_input_length = 300

overlap = {"side":"both", "number": 50}

In [49]:
train_set = hyp_train._chunk(200, overlap=overlap)
train_index, train_text = zip(*train_set)

#Creates dictionary for train_index and train_text
train_index = {a:x for a,x in enumerate(train_index)}
train_text = {a:x for a,x in enumerate(train_text)}

In [50]:
test_set = hyp_test._chunk(200, overlap=overlap)
test_index, test_text = zip(*test_set)

#Creates dictionary for test_index and test_text

test_index = {a:x for a,x in enumerate(test_index)}
test_text = {a:x for a,x in enumerate(test_text)}

In [51]:
#Creates dictionary of labels
train_chunked_label = {i:hyperpartisan_train['labels'][x] for i,x in train_index.items()}

In [52]:
test_chunked_label = {i:hyperpartisan_test['labels'][x] for i,x in test_index.items()}

In [53]:
train_set = {"train" : {"text": train_text.values(), "doc_id": train_index.values(), "labels": train_chunked_label.values()}}
test_set =  {"test" : {"text": test_text.values(), 'doc_id': test_index.values(), 'labels': test_chunked_label.values()}}

In [54]:
import pandas as pd

a = pd.DataFrame(train_set['train'])
b = pd.DataFrame(test_set['test'])

In [55]:
#Modify types
a['text'] = a['text'].astype(str)
a['labels'] = a['labels'].astype(int)

b['text'] = b['text'].astype(str)
b['labels'] = b['labels'].astype(int)

In [58]:
from datasets import Dataset

In [59]:
train_set = Dataset.from_pandas(a)
test_set = Dataset.from_pandas(b)

In [60]:
train_set = train_set.shuffle(seed = 32)
#test_set = test_set.shuffle(seed = 32)

In [63]:
del hyperpartisan_valid

In [61]:
train_set = train_set.map(tokenize)

  0%|          | 0/47618 [00:00<?, ?ex/s]

In [62]:
test_set = test_set.map(tokenize)

  0%|          | 0/47618 [00:00<?, ?ex/s]

In [69]:
from sklearn.model_selection import KFold

In [77]:
fold = KFold(n_splits = 5)
this_set = fold.split(train_set['text'])

fold_train = []
fold_eval = []

for i,j in this_set:
  fold_train.append(train_set.select(i))
  fold_eval.append(train_set.select(j))

In [ ]:
#for multi-class classification
#labels = hyperpartisan_train["label"]
#num_labels = len(labels)
#labels2id, id2labels = dict(),dict()
#for i,label in enumerate(labels):
    #labels2id[label] = i
    #id2labels[i] = label

def bert_init():
  return (BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, max_length = max_input_length))

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2, max_length = max_input_length)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #to pad sequences

#default training arguments 
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=16, 
                                  evaluation_strategy= "epoch", per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)

#fine-tuning model
trainer = Trainer(model = model,
              args = training_args,
              train_dataset= fold_train[1],
              eval_dataset= fold_eval[1],
              tokenizer= tokenizer,
              data_collator=data_collator,
              compute_metrics= compute_metrics)

#searching for best hyperparameters
trainer.train()

In [79]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: doc_id, text. If doc_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9524
  Batch size = 16


Trainer is attempting to log a value of "{'f1': 0.8847164625875646}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8849826713620268}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.885132297354053}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.7012601494789124,
 'eval_f1': {'f1': 0.8847164625875646},
 'eval_precision': {'precision': 0.8849826713620268},
 'eval_recall': {'recall': 0.885132297354053},
 'eval_runtime': 66.9252,
 'eval_samples_per_second': 142.308,
 'eval_steps_per_second': 8.905,
 'epoch': 5.0}

In [84]:
result = trainer.predict(test_set)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: doc_id, text. If doc_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 47618
  Batch size = 16


In [85]:
results_output = list(np.argmax(result.predictions, axis = 1))

In [ ]:
results_output

In [87]:
orig = list(result.label_ids)
orig

array([1, 1, 1, ..., 0, 0, 0])

In [2]:
import pickle

In [90]:
with open("hyperpartisan_chunk_200_50.pkl", "wb") as out_1:
  pickle.dump(results_output, out_1)

with open("hyperpartisan_chunk_orig_200_50.pkl", "wb") as out_2:
  pickle.dump(orig, out_2)

In [7]:
with open("hyperpartisan_chunk_200_100.pkl", "rb") as in_1:
  chunk_res = pickle.load(in_1)

with open("hyperpartisan_chunk_orig_200_100.pkl", "rb") as in_2:
  chunk_orig = pickle.load(in_2)

In [8]:
correct = []; incorrect = []; incorrect_1 = []

for a,(b,c) in enumerate(zip(chunk_res,chunk_orig)):
    if b ==c :
        correct.append(a)
    else:
        incorrect_1.append(a)

In [9]:
incorrect_1

[12,
 182,
 194,
 222,
 223,
 248,
 276,
 302,
 324,
 365,
 387,
 424,
 446,
 452,
 474,
 475,
 562,
 600,
 658,
 661,
 662,
 697,
 707,
 728,
 733,
 746,
 776,
 813,
 830,
 882,
 896,
 988,
 1009,
 1049,
 1056,
 1094,
 1115,
 1124,
 1147,
 1185,
 1195,
 1233,
 1342,
 1346,
 1431,
 1451,
 1471,
 1478,
 1577,
 1578,
 1606,
 1627,
 1630,
 1634,
 1661,
 1690,
 1693,
 1707,
 1723,
 1735,
 1766,
 1819,
 1827,
 1841,
 1852,
 2106,
 2180,
 2233,
 2244,
 2245,
 2250,
 2255,
 2258,
 2295,
 2306,
 2433,
 2486,
 2525,
 2542,
 2552,
 2688,
 2738,
 2823,
 2827,
 2830,
 2865,
 2884,
 2898,
 2997,
 3109,
 3161,
 3163,
 3169,
 3170,
 3210,
 3211,
 3252,
 3302,
 3368,
 3451,
 3486,
 3597,
 3603,
 3606,
 3641,
 3657,
 3668,
 3696,
 3736,
 3935,
 3942,
 3956,
 3977,
 3990,
 4024,
 4036,
 4065,
 4074,
 4103,
 4115,
 4193,
 4220,
 4259,
 4278,
 4295,
 4302,
 4316,
 4323,
 4334,
 4351,
 4360,
 4373,
 4380,
 4410,
 4513,
 4531,
 4622,
 4697,
 4726,
 4769,
 4771,
 4775,
 4788,
 4804,
 4873,
 4939,
 4947,
 505

In [6]:
incorrect

[182,
 194,
 248,
 276,
 302,
 324,
 365,
 387,
 424,
 443,
 452,
 474,
 475,
 478,
 534,
 562,
 600,
 634,
 657,
 661,
 662,
 697,
 707,
 724,
 728,
 732,
 733,
 746,
 776,
 797,
 813,
 830,
 882,
 896,
 988,
 1009,
 1049,
 1051,
 1056,
 1094,
 1124,
 1147,
 1177,
 1185,
 1195,
 1233,
 1301,
 1335,
 1340,
 1342,
 1346,
 1354,
 1451,
 1471,
 1478,
 1497,
 1577,
 1578,
 1603,
 1606,
 1627,
 1630,
 1634,
 1641,
 1661,
 1690,
 1693,
 1707,
 1723,
 1724,
 1735,
 1766,
 1771,
 1814,
 1819,
 1827,
 1841,
 1852,
 1896,
 2106,
 2180,
 2216,
 2233,
 2244,
 2250,
 2258,
 2292,
 2306,
 2412,
 2433,
 2486,
 2525,
 2542,
 2552,
 2654,
 2688,
 2738,
 2772,
 2827,
 2842,
 2865,
 2884,
 2898,
 2949,
 3026,
 3079,
 3109,
 3161,
 3163,
 3169,
 3170,
 3190,
 3210,
 3222,
 3252,
 3302,
 3340,
 3349,
 3368,
 3384,
 3451,
 3486,
 3597,
 3603,
 3606,
 3641,
 3657,
 3668,
 3689,
 3696,
 3701,
 3736,
 3801,
 3824,
 3925,
 3929,
 3942,
 3956,
 3977,
 3990,
 4024,
 4036,
 4065,
 4074,
 4103,
 4193,
 4220,
 4259,

In [ ]:
#display result 
#display confusion matrix